In [1]:
import csv # for writing dataframes to csv
import random # for making a random choice
import os # for scanning directories
import itertools
import string # for generating strings

import kintypes as kt # bringing large lists of kin types into the namespace
import math # for calculating logs
import pandas as pd

# Internal co-selection

Internal co-selection refers to the tendency for kinship systems to have cross-generational consistency in the terminological distinctions or mergers that are made. That is, if your parents' elder brothers share a kin term, then so too will their children. If your parents' sisters are distinguished from your parents' brothers, so too will their children be distinguished. We can test the robustness of this tendency using our frankenlanguages, to see whether internal co-selection occurs at a higher rate than chance.

We will measure internal co-selection in terms of the **mutual information** between Generation N and Generation N+1 in a particular kinship system. That tells us how much information can be gained from one generation by observing the other - we can think of this as the benefit of internal co-selection. That is, we need to work out the conditional entropy between every possible pair of parent and child terms, and the entropy over an entire generation. This will tell us how much information is shared across the two generations; or how much we can predict about one generation given the other.

To do this, we need to do the following:

* Get a list of parent-child pairs for each language.
* Work out the conditional probabilities for each pair.
* Work out the probabilities of each individual term in a generation.
* Calculate the entropy of 2 and 3.
* Calculate the mutual information of the system.

Luckily, we can re-use some of the infrastructure we already have. For ease, I will write out again the functions that extract kin terms from a kinbank file.

In [2]:
# to get a list of all the kinbank filenames

def get_kb_files():
    files = []
    path = '../languages/kinbank'
    directory = os.scandir(path)
    for file in directory:
        files.append(file.name)
    return files

In [3]:
# to pick a file at random

def random_language(all_data):
    language = random.choice(all_data)
    # print(language)
    return language

In [4]:
# to extract kin terms from one of those files

def get_kin_terms(filepath):
    kin_system = {}
    with open(filepath, encoding='utf8') as f:
        csv_reader = csv.DictReader(f)
        next(csv_reader) # to skip the header row
        for line in csv_reader:
            kin_type = line['parameter']
            kin_term = line['word']
            kin_system[kin_type] = kin_term
    return kin_system

For testing purposes throughout this notebook, let's pick a random language and extract its kin terms.

In [5]:
all_kb_files = get_kb_files()

In [20]:
random.seed(47)
file = random_language(all_kb_files)
filepath = '../languages/kinbank/'

l = get_kin_terms(filepath + file)

print(file,l)

Nogai_noga1249.csv {'mG': 'karɨndas', 'mB': 'adanas', 'mZ': 'karɨndas', 'myB': 'ini', 'myZ': 'siŋli', 'mF': 'ata', 'mM': 'ana', 'mC': 'tuwgailar', 'mS': 'kede', 'mD': 'kɨz', 'mFF': 'atay', 'mFM': 'tetey', 'mMF': 'atay', 'mMM': 'tetey', 'mSS': 'yiyen', 'mSD': 'yiyеn kɨz', 'mDS': 'yiyen', 'mDD': 'yiyеn kɨz', 'mFB': 'akay', 'mFZ': 'abay', 'mMB': 'nagaš akay', 'mMZ': 'abay', 'mBS': 'yiyen', 'mBD': 'yiyеn kɨz', 'mZS': 'yiyen', 'mZD': 'yiyеn kɨz', 'mFZD': 'bölе', 'mFBD': 'bölе', 'mMBD': 'bölе', 'mMZD': 'bölе', 'mFBS': 'bölе', 'mFZS': 'bölе', 'mMBS': 'bölе', 'mMZS': 'bölе', 'mH': 'kiew', 'mW': 'pišе', 'mHF': 'kaynata', 'mHM': 'kaynana', 'mWF': 'kaynata', 'mWM': 'kaynana', 'mSW': 'kеšеk', 'mDH': 'kiеw', 'meB': 'adanas', 'meZ': 'karɨndas', 'mFeB': 'akay', 'mFyB': 'akay', 'mFeZ': 'abay', 'mFyZ': 'abay', 'mMeZ': 'abay', 'mMyZ': 'abay', 'mMeB': 'nagaš akay', 'mMyB': 'nagaš akay', 'meBS': 'yiyen', 'myBS': 'yiyen', 'meBD': 'yiyеn kɨz', 'myBD': 'yiyеn kɨz', 'meZS': 'yiyen', 'myZS': 'yiyen', 'meZD': '

## Getting the pairs

~~The first thing we need to do is write a function that takes a dictionary of kin terms like `l`, and outputs a list of the relevant terms for each generation. We also need a function that pairs up those terms into parent-child pairs. In `kintypes`, we have a list of pairs of codes for parent and child terms, so we just need to cross reference these.~~

To avoid weird gaps, we're going to slightly simplify things! First, we're going to extract the relevant pairs of terms from a dictionary of kin terms - e.g. for English, it extracts the pair `('uncle','cousin')`. Notably, our function checks whether **both** of the relevant terms exist in the dictionary before trying to extract them! Then, we'll split these pairs in half to get the list of terms in each generation. This way, any gaps in the kinbank data - e.g. the term for 'uncle' is there, but the term for 'cousin' is not - cause us less strife, at the expense of a little bit of accuracy.

In [6]:
def get_pairs(ks):
    pairs_of_terms = []
    placeholder = []
    
    for pair in kt.ics_pairs:
        if pair[0] in ks and pair[1] in ks:
            pairs_of_terms.append((ks[pair[0]],ks[pair[1]]))
            placeholder.append(pair)
            
    #if len(pairs_of_terms) > 40: # if at least 1/4 of the relevant pairs are present      
    return pairs_of_terms

In [ ]:
l_pairs = get_pairs(l)

print(l_pairs)

Now we have a way to get a list of pairs, let's split them up into `GN` for Ego's generation and `GN1` for ego's parents' generation.

In [7]:
def split_generations(pairs):
    GN = []
    GN1 = []
    for pair in pairs:
        GN.append(pair[1])
        GN1.append(pair[0])
    
    return GN,GN1

In [ ]:
l_GN, l_GN1 = split_generations(l_pairs)

print(l_GN,l_GN1)

## Calculating probabilities

To calculate entropy of one generation of a kinship system, we will need to calculate the probability distribution of the terms in that generation. To calculate conditional entropy, we will need a probability distribution for all the pairs of terms in the kinship system. Below is a function that takes a list of elements and calculates the probability of each of those elements w.r.t. the others - those elements can be terms or pairs.

In [8]:
def probability(term: str, generation: list) -> float:
    print(generation.count(term),len(generation),generation.count(term)/len(generation))
    return generation.count(term)/len(generation)

In [ ]:
l_term = random.choice(l_GN)

print(l_term, probability(l_term,l_GN))

We can use this function to calculate the conditional probability of each term given the term it is paired with: the probability of the pair (given all the possible pairs) over the probability of the "given" term. Let's do this "bottom-up" for ease - so for English, calculating the probability of *uncle* given *cousin*. Since mutual information is not directional, we can make this arbitrary choice without repercussions later.

~~Reader, there were repercussions. In my first pass, I got it the wrong way around! Ack.~~

In [9]:
def joint_probability(term1: str, term2: str, pairs: list):
    pair = (term1,term2)
    return pairs.count(pair)/len(pairs)

In [ ]:
l_pair = random.choice(l_pairs)

print(l_pairs,l_pair, joint_probability(l_pair[0],l_pair[1],l_pairs))

## Calculating entropy and mutual information

Now that we have a way to extract probability distributions from a kinship system, we can feed that in to a function that calculates entropy. The entropy scores can then be fed to a function that calculates mutual information (equations to follow).

In [10]:
def entropy(generation):
    entropy = 0
    for term in set(generation):
        p = probability(term,generation)
        #print(term, p)
        print(term,p*math.log(p))
        entropy += p*math.log(p)
    return -entropy

In [ ]:
entropy(l_GN)

We need a second function to calculate conditional entropy, which is slightly more complex. Here, we calculate the probability of B and A multiplied by the probability of B and A over B. We can re-use our `calculate_probs()` function to calculate to the probability of each pair (p(B and A)).

In [11]:
def conditional_entropy(g1,g2,pairs):
    entropy = 0
    for x,y in set(pairs): # parent,child
        p_xy = joint_probability(x,y,pairs)
        p_y = probability(y,g1)
        print(x,y, 'p(', x, '|', y,') = ', p_xy/p_y, 'p(y) = ', p_y)
        if p_xy > 0 and p_y > 0:
            entropy += p_xy * math.log(p_xy/p_y)
    return -entropy

In [ ]:
conditional_entropy(l_GN,l_GN1,l_pairs)

These two values can then be used to calculate mutual information. Because of our 'top-down' approach, this will be equal to the entropy of `l`'s Generation N minus the conditional entropy of `l`.

In [12]:
def mutual_information(e,ce):
    return e - ce

## Getting neat and tidy

Now let's write a function that wraps up aaaaall of the above neatly. We want to return entropy, conditional entropy, and mutual information individually so we can keep track of these values for the analysis later. This will be particularly useful for the simulation work coming up next, where we may want to visualise mutual information by entropy at one generation.

First though, we want to write a flag function that tells us if something's wrong with the calculations. MI cannot be less than 0, so if this happens, we want all of the relevant information to help us solve the problem to be printed.

In [ ]:
# def mi_check(pairs,GN,GN1,probs):
#     #print('STOP: SOMETHING IS VERY WRONG')
#     p_pairs = calculate_probs(pairs)
#     cp = calculate_conditional_probs(pairs,GN)
#     for term,prob in zip(set(GN1),probs):
#         print('term: ', term, 'prob:', prob)
#     for pair in set(pairs):
#         index = list(set(pairs)).index(pair)
#         print('pair: ', pair, 'prob:', p_pairs[index], 'prob ' + pair[1] +'|' + pair[0], cp[index])
#     print('probability distribution of GN1: ', set(GN1), probs)
#     print('probability distribution of pairs: ', set(pairs), calculate_probs(pairs))
#     print('conditional probability of pairs: ', set(pairs), calculate_conditional_probs(pairs,GN))

In [29]:
def mi_of_system(ks):
    pairs = get_pairs(ks)
    if pairs:
        g1,g2 = split_by_gen(ks)

        h = entropy(g2)
        h_xy = conditional_entropy(g1,g2,pairs)
        mi = mutual_information(h,h_xy)
        if mi < 0:
            print('STOP, SOMETHING IS VERY WRONG')
        return h,h_xy,mi
    else:
        return 'NA','NA','NA'

In [ ]:
mi_of_system(l)

To make sure that our function (with all its hand-calculated components) is working effectively, we can write a quick function that uses a pre-existing MI calculator `mutual_info_score()` from `sklearn` to double check our results. It's possible that they will yield slightly different scores (given that our function knows which terms pair together with what frequency, which `mutual_info_score()` doesn't know), but it is still a good baseline to work from. Ideally, we would like to use the hand-calculations so that we can record the entropy and conditional entropy associated with each kinship system alongside the MI.

In [ ]:
from sklearn.metrics import mutual_info_score

def easy_mi(ks):
    pairs = get_pairs(ks)
    if pairs:
        GN,GN1 = split_generations(pairs)
        #print(pairs)
        #GN,GN1 = split_by_gen(ks)
        # pair_codes = code_pairs(pairs)
        if pairs: # if there are any matching pairs in this language
            mi = mutual_info_score(GN,GN1)
            #mi2 = mutual_info_score(GN1,GN)
            if mi < 0:
                print(mi)
                print('SOMETHING IS VERY WRONG')
                #print(GN,GN1)
            return mi

In [ ]:
easy_mi(l)

In [ ]:
counter = 0

for file in all_kb_files:
    print(file)
    ks = get_kin_terms(filepath + file)
    print(mi_of_system(ks), easy_mi(ks))
    if mi_of_system(ks) != None:
        print('OK')
        counter += 1
        
print(counter)

## Simulating kinship systems

We now have a function that can take a kinship system and output a mutual information score for the terms in Ego and Ego's parents' generations. However, while running this function on all the world's languages can tell us something about the distribution of internal co-selection across the world, it doesn't tell us whether the world's languages co-select at a greater rate than chance. If we want to argue that internal co-selection is a product of cultural evolution, we need to dispel the possibility that it occurs by chance.

To get an idea of what chance is, we need to simulate some truly randomly generated systems. We can compare the MI of these simulations to the real languages to see whether the real languages have significantly greater mutual information between generations.

Because MI is dependent on the amount of variation in a language, we need a simulation which maintains the number of terms while randomising which child terms pair with which parent terms. So we will take each language in our data, and randomly scramble which terms go with which types (within generation).

To do this, we need:

* A function to extract the kinship system of a language from kinbank (check!)
* A function that filters the two generations we are interested in (sort of check!)
* A function that takes all of the terms in that system and randomly reassigns them to types.
* A function that repeats this process a bunch of times.

Let's extract a new random language from kinbank to work with.

In [21]:
random.seed(101)

file = 'Koya_koya1251.csv'
l2 = get_kin_terms(filepath + file)

print(file,l2)

Koya_koya1251.csv {'mZ': 'tudal', 'mF': 'bābo', 'mM': 'yāyo', 'mS': 'maru', 'mD': 'mayāḍ', 'mFF': 'dado', 'mFM': 'bāpi', 'mMM': 'kakko', 'mFB': 'pépe', 'mFZ': 'poyé', 'mMB': 'māmā', 'mMeZ': 'pedi', 'mMyZ': 'kuci', 'mBS': 'maru', 'mBD': 'mayyāḷ', 'mZS': 'koḍiār', 'mZD': 'mayyal', 'mFZD': 'pekki', 'mFBD': 'piki', 'mMBD': 'pekki', 'mMZD': 'piki', 'mFBS': 'tammuḍu', 'mFZS': 'yeruḷ', 'mMBS': 'yeruḷ', 'mMZS': 'tammuḍu', 'mH': 'mudiāl', 'mW': 'mutte', 'mHF': 'māmāl', 'mHM': '0', 'mWF': 'māmāl', 'mWM': '0', 'mBW': 'eravu-pekkal', 'mZH': 'ende-thor', 'mWB': 'erupekka', 'mWZ': 'eru-pitti', 'mHB': 'erul', 'mSW': 'kodiaḷ', 'mDH': 'aṇḍe', 'mFZH': 'māmā', 'mFBW': 'kuci', 'mMZH': 'kākā', 'mMBW': 'poyé', 'fB': 'dādālo', 'fZ': 'tudal', 'fF': 'bābo', 'fM': 'yāyo', 'fS': 'maru', 'fD': 'mayāḍ', 'fFF': 'dado', 'fFM': 'bāpi', 'fMM': 'kakko', 'fFB': 'pépe', 'fFZ': 'poyé', 'fMB': 'māmā', 'fMeZ': 'pedi', 'fMyZ': 'kuci', 'fBS': 'koḍiār', 'fBD': 'mayyal', 'fZS': 'maru', 'fZD': 'peki', 'fFZD': 'pekki', 'fFBD': 'p

Now we have a kinship system, we can write a function to filter out the relevant terms into two lists - one for Ego's generation and one for Ego's parents' generation. In `kintypes`, we have lists of kin types for each generation which can help us with this task.

In [15]:
def split_by_gen(ks):
    g1 = {}
    g2 = {}
    for entry in ks:
        if entry in kt.generation_n:
            g1[entry] = ks[entry]
        elif entry in kt.generation_n1:
            g2[entry] = ks[entry]
        else:
            pass
    
    return g1,g2

In [ ]:
g1,g2 = split_by_gen(l2)

Now we can randomly redistribute the keys in these dictionaries among the values in these dictionaries and combine them, giving us a simulated kinship system that maintains the amount of variation within generations while completely randomising the information shared between those two generations.

In [16]:
def randomise_generation(g):
    sim_g = {}
    terms = list(g.values())
    types = list(g.keys())
    random.shuffle(terms)
    
    for i in range(len(g)):
        random_term = terms[i]
        kintype = types[i]
        sim_g[kintype] = random_term
        
    return sim_g
        

In [ ]:
randomise_generation(g1)

And finally a function that splits by generation, randomises the two generations, and sticks them back together:

In [34]:
def shuffle_ks(ks):
    gN, gN1 = split_by_gen(ks)
    gN_sim = randomise_generation(gN)
    gN1_sim = randomise_generation(gN1)
    
    print(len(gN),len(gN_sim),len(gN1),len(gN1_sim))
    
    print(len({**gN_sim, **gN1_sim}))
    
    return {**gN_sim, **gN1_sim}

In [52]:
l2_sim = shuffle_ks(l2)

87 87 26 26
113


In [53]:
a,b = split_generations(get_pairs(l2))
print(len(a),len(b))

c,d = split_generations(get_pairs(l2_sim))
print(len(c),len(d))

86 86
85 85


Now let's check the MI of our random language `l` and compare it against our simulation of `l`, `l_sim`. We can see that their entropy is identical (as to be expected as the amount of variation is the same) but their conditional entropy varies. In this instance we get a smaller MI for the simulation than for the real language.

In [ ]:
print(mi_of_system(l2),easy_mi(l2))

In [ ]:
print(mi_of_system(l2_sim),easy_mi(l2_sim))

Now we can create a function that takes a language, randomises it and calculates its MI a specified number of times, and saves all of those values to a dataframe to be analysed later.

In [ ]:
def write_results(sim: str,mi: float, e: float, ce: float,system: dict):
    results = {}
    results['simulation_number'] = sim
    results['mutual_information'] = mi
    results['entropy'] = e
    results['conditional_entropy'] = ce
    for i in system:
        results[i] = system[i]
    return results

In [ ]:
def simulation(file,times):
    filepath = '../languages/kinbank/'
    language = file[:-13]
    ks = get_kin_terms(filepath + file)
    print(ks)
    df = []
    print(file)
    for i in range(times):
        shuffled_system = shuffle_ks(ks)
        #mi = easy_mi(shuffled_system)
        e,ce,mi = mi_of_system(shuffled_system)
        
        if mi:
            results = write_results(str(i),mi,e,ce,shuffled_system)
            df.append(results)
    
    pd.DataFrame(df).to_csv('../data/raw/ics_' + language + '.csv',index=False)
        
    return pd.DataFrame(df)
    

And we can test that with just a few rounds of the simulation for our random language before running the full simulation on all the kinbank files.

In [ ]:
simulation(file,10)

In [ ]:
def full_ics_simulation(all_files):
    for file in all_files:
        simulation(file,1000)

In [ ]:
full_ics_simulation(all_kb_files)

In [ ]:
simulation('Bau-Jagoi_Bidayuh_baub1235.csv',10)

## Testing, testing

And now to test until we brute force our way to working code!

In [ ]:
hindi = get_kin_terms(filepath + 'Hindi_hind1269.csv')

#hindi_sim = simulation('Hindi_hind1269.csv',1000)

hindi_ics = mi_of_system(hindi)

hindi_ics

In [ ]:
english = get_kin_terms(filepath + 'English_stan1293.csv')

In [ ]:
aguaruna = get_kin_terms(filepath + 'Aguaruna_agua1253.csv')

### Testing hand-made languages

What if our language co-selects perfectly?

In [ ]:
ics_lang = {
    'mM': 'mmm',
    'mF': 'fff',
    'meB': 'mmma',
    'myB': 'mmma',
    'meZ': 'fffa',
    'myZ': 'fffa',
    'mMeB': 'aaa',
    'mMeZ': 'bbb',
    'mFeB': 'ccc',
    'mFeZ': 'ddd',
    'mMeBS': 'aaas',
    'mMeZS': 'bbbs',
    'mFeBS': 'cccs',
    'mFeZS': 'ddds',
    'mMeBD': 'aaad',
    'mMeZD': 'bbbd',
    'mFeBD': 'cccd',
    'mFeZD': 'dddd',
    'mMyB': 'eee',
    'mMyZ': 'fff',
    'mFyB': 'ggg',
    'mFyZ': 'hhh',
    'mMyBS': 'eees',
    'mMyZS': 'fffs',
    'mFyBS': 'gggs',
    'mFyZS': 'hhhs',
    'mMyBD': 'eeed',
    'mMyZD': 'fffd',
    'mFyBD': 'gggd',
    'mFyZD': 'hhhd'
}

mi_of_system(ics_lang)
#easy_mi(ics_lang)

Does that change if the son and daughter terms are the same?

In [ ]:
ics_lang_2 = {
    'mM': 'mmm',
    'mF': 'fff',
    'meB': 'mmma',
    'myB': 'mmma',
    'meZ': 'mmma',
    'myZ': 'mmma',
    'mMeB': 'aaa',
    'mMeZ': 'bbb',
    'mFeB': 'ccc',
    'mFeZ': 'ddd',
    'mMeBS': 'aaac',
    'mMeZS': 'bbbc',
    'mFeBS': 'cccc',
    'mFeZS': 'dddc',
    'mMeBD': 'aaac',
    'mMeZD': 'bbbc',
    'mFeBD': 'cccc',
    'mFeZD': 'dddc',
    'mMyB': 'eee',
    'mMyZ': 'fff',
    'mFyB': 'ggg',
    'mFyZ': 'hhh',
    'mMyBS': 'eeec',
    'mMyZS': 'fffc',
    'mFyBS': 'gggc',
    'mFyZS': 'hhhc',
    'mMyBD': 'eeec',
    'mMyZD': 'fffc',
    'mFyBD': 'gggc',
    'mFyZD': 'hhhc'
}

mi_of_system(ics_lang_2)
#easy_mi(ics_lang_2)

And what about a non co-selecting language?

In [ ]:
bad_ics_lang = {
    'mM': 'mmm',
    'mF': 'fff',
    'meB': 'aaad',
    'myB': 'dddd',
    'meZ': 'cccd',
    'myZ': 'ddds',
    'mMeB': 'aaa',
    'mMeZ': 'bbb',
    'mFeB': 'ddd',
    'mFeZ': 'ddd',
    'mMeBS': 'eed',
    'mMeZS': 'aaas',
    'mFeBS': 'aaas',
    'mFeZS': 'ddds',
    'mMeBD': 'aaad',
    'mMeZD': 'aaad',
    'mFeBD': 'dddd',
    'mFeZD': 'aaad',
    'mMyB': 'eee',
    'mMyZ': 'fff',
    'mFyB': 'ddd',
    'mFyZ': 'hhh',
    'mMyBS': 'aaas',
    'mMyZS': 'eee',
    'mFyBS': 'aaas',
    'mFyZS': 'gggs',
    'mMyBD': 'eeed',
    'mMyZD': 'aaad',
    'mFyBD': 'aaad',
    'mFyZD': 'gggd'
}

mi_of_system(bad_ics_lang)
#easy_mi(bad_ics_lang)

It's not super intuitive to me that the "worst" languages are those with entropy 0 in one of the generations. A system like English is at least consistent - if the motivation for ICS is that it increases simplicity, then this seems entirely opposite to what we want!

## What about randomly generated languages?

Let's write a function to randomly generate a language so we can get a feel for what good / bad mutual information scores look like.

In [ ]:
def generate_language(kin_types):
    language = {}
    words = []
    letters = string.ascii_lowercase
    
    for i in range(len(kin_types)):
        word = ''.join(random.choice(letters) for i in range(4))
        words.append(word)
        
    print(words)
        
    for kt in kin_types:
        term = random.choice(words)
        language[kt] = term
        
    return language

In [ ]:
x = generate_language(ics_lang.keys())

get_pairs(x,kt.ics_pairs)
      
#calculate_ics(x)

In [ ]:
length = ['mMeB','mMeZ','mFeB','mFeZ','mMeBS','mMeZS','mFeBS','mFeZS']

r_l = generate_language(length)

r_p = get_pairs(r_l,kt.ics_pairs)

#calculate_ics(r_l)

In [ ]:
#full_ics_simulation(all_kb_files)

In [ ]:
agob = get_kin_terms(filepath + 'Agob-Ende-Kawam_agob1244.csv')

print(get_pairs(agob,kt.ics_pairs))

mi_of_system(agob)

In [ ]:
def calculate_ics(ks):
    pairs = get_pairs(ks,kt.ics_pairs)
    GN,GN1 = split_generations(pairs)
    probs = calculate_probs(GN1)
    e = entropy(probs)
    ce = conditional_entropy(pairs,GN)
    mi = mutual_information(e,ce)
    for pair in set(pairs):
        print(pair,pairs.count(pair))
    for term in set(GN):
        print(term,GN.count(term))
    for term in set(GN1):
        print(term, GN1.count(term))
    
    if mi < 0:
        print('mi = ', mi, 'ce = ', ce, 'e = ', e)
        mi_check(pairs,GN,GN1,probs)
        
    return mi,ce,e

In [ ]:
len(kt.ics_pairs)

164/4